About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:


        1: Fake news
        0: real News

Importing the Dependencies

In [51]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [52]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

printing the stopwords in English

In [53]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Data Pre-processing

loading the dataset to a pandas DataFrame

In [54]:
news_dataset = pd.read_csv('G:\\Machine Learning\\Fake_News_Prediction\\train.csv')

In [55]:
news_dataset.shape

(20800, 5)

Print the first 5 rows of the dataframe

In [56]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


Counting the number of missing values in the dataset

In [57]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

Replacing the null values with empty string

In [58]:
news_dataset=news_dataset.fillna('')

Merging the author name and news title

In [59]:
news_dataset['content'] = news_dataset['author']+''+news_dataset['title']

In [60]:
print(news_dataset['content'])

0        Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1        Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2        Consortiumnews.comWhy the Truth Might Get You ...
3        Jessica Purkiss15 Civilians Killed In Single U...
4        Howard PortnoyIranian woman jailed for fiction...
                               ...                        
20795    Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796    Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat...
20797    Michael J. de la Merced and Rachel AbramsMacy’...
20798    Alex AnsaryNATO, Russia To Hold Parallel Exerc...
20799               David SwansonWhat Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


Separating the data & label

In [61]:
x = news_dataset['content'].values
y = news_dataset['label'].values

In [62]:
print(x)

['Darrell LucusHouse Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'
 'Daniel J. FlynnFLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'
 'Consortiumnews.comWhy the Truth Might Get You Fired' ...
 'Michael J. de la Merced and Rachel AbramsMacy’s Is Said to Receive Takeover Approach by Hudson’s Bay - The New York Times'
 'Alex AnsaryNATO, Russia To Hold Parallel Exercises In Balkans'
 'David SwansonWhat Keeps the F-35 Alive']


In [63]:
y

[1 0 1 ... 0 1 1]


In [64]:
y.shape

(20800,)

Stemming:

Stemming is the process of reducing a word to its Root word

example: actor, actress, acting --> act

In [65]:
port_stem = PorterStemmer()

In [66]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [67]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [68]:
print(news_dataset['content'])

0        darrel lucushous dem aid even see comey letter...
1        daniel j flynnflynn hillari clinton big woman ...
2                consortiumnew comwhi truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoyiranian woman jail fiction unpub...
                               ...                        
20795    jerom hudsonrapp trump poster child white supr...
20796    benjamin hoffmann f l playoff schedul matchup ...
20797    michael j de la merc rachel abramsmaci said re...
20798    alex ansarynato russia hold parallel exercis b...
20799                        david swansonwhat keep f aliv
Name: content, Length: 20800, dtype: object


In [69]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [70]:
print(X)

['darrel lucushous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynnflynn hillari clinton big woman campu breitbart'
 'consortiumnew comwhi truth might get fire' ...
 'michael j de la merc rachel abramsmaci said receiv takeov approach hudson bay new york time'
 'alex ansarynato russia hold parallel exercis balkan'
 'david swansonwhat keep f aliv']


In [71]:
print(Y)

[1 0 1 ... 0 1 1]


Converting the textual data to numerical data

In [72]:
vectorizer = TfidfVectorizer()
vectorizer.fit(x)

x=vectorizer.transform(x)

In [73]:
print(x)

  (0, 34402)	0.17918739747269477
  (0, 33460)	0.2620405036866357
  (0, 33092)	0.34354025525372645
  (0, 28544)	0.22370120836039492
  (0, 19040)	0.35769014842767877
  (0, 18475)	0.24826788911182338
  (0, 16508)	0.20476648736259792
  (0, 16295)	0.1536111400920685
  (0, 10594)	0.23028973074454998
  (0, 8620)	0.24398932179297475
  (0, 8241)	0.2740085657861192
  (0, 7840)	0.2951614509491496
  (0, 6517)	0.20565002646673405
  (0, 5558)	0.3015240959717515
  (0, 1181)	0.26093253981852516
  (1, 35080)	0.3619575823267221
  (1, 22730)	0.19046856538729842
  (1, 14631)	0.23703745360274664
  (1, 11652)	0.5156618392052476
  (1, 7777)	0.313608220062674
  (1, 6219)	0.23118052033472167
  (1, 5116)	0.4508844821928066
  (1, 4420)	0.18290583060889654
  (1, 3621)	0.3493832273397924
  (2, 35465)	0.25674258355397445
  :	:
  (20797, 20409)	0.3114229497461508
  (20797, 17935)	0.21981444256219335
  (20797, 16233)	0.126331449970448
  (20797, 15082)	0.2922971928123606
  (20797, 7967)	0.20435849370299847
  (20797, 4

Splitting the dataset to training & test data

In [74]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

Training the Model: Logistic Regression

In [75]:
model = LogisticRegression()

In [76]:
model.fit(x_train,y_train)

LogisticRegression()

Evaluation

Accuracy score on the training data

In [77]:
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction,y_test)

In [78]:
print("Accuracy score on the test data : ",test_data_accuracy)

Accuracy score on the test data :  0.9548076923076924


Making a Predictive System

In [79]:
x_news = x_test[6]

prediction = model.predict(x_news)
print(prediction)

if(prediction[0]==0):
    print("The news is Real.")
else:
    print("The news is Fake")

[0]
The news is Real.


In [80]:
print(y_test[6])

0
